# Market Structure: Swings & Trend Regime Visualization

This notebook visualizes the first set of market structure signals:
1. Swing Highs & Lows
2. Trend Regime

The goal is to qualitatively inspect the signals to ensure they are being detected logically on the price chart.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Ensure the project root is in the Python path
ROOT = Path().resolve().parents[2]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from finantradealgo.features.feature_pipeline import (
    build_feature_pipeline_from_system_config,
    FeaturePipelineConfig
)

# Configure plots
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (20, 12)

print("Imports complete and environment set up.")

In [ ]:
# --- Configuration ---
SYMBOL = "BTCUSDT"
TIMEFRAME = "15m"

# --- Run the feature pipeline ---
print("Running feature pipeline for market structure...")
pipeline_cfg = FeaturePipelineConfig(
    use_market_structure=True,
    # Disable other features to speed up this run
    use_microstructure=False,
    use_base=False,
    use_ta=False,
    use_candles=False,
    use_osc=False,
    use_htf=False,
    use_external=False,
    use_rule_signals=False,
    drop_na=False, # Keep all rows to align with original data
)

df, _ = build_feature_pipeline_from_system_config(
    pipeline_cfg=pipeline_cfg, 
    symbol=SYMBOL, 
    timeframe=TIMEFRAME
)
df = df.set_index(pd.to_datetime(df['timestamp'])).drop('timestamp', axis=1)
print(f"Pipeline complete. DataFrame shape: {df.shape}")
df.tail()

In [ ]:
# --- Visualization ---
df_slice = df.last('120D') # Use a slice for better visibility

fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})

# Panel 1: Price and Swings
ax1 = axes[0]
ax1.plot(df_slice.index, df_slice['close'], label='Close', color='gray', alpha=0.8)
ax1.set_title(f'{SYMBOL} Price with Swing Points')

# Find swing points to plot
swing_highs = df_slice[df_slice['ms_swing_high'] == 1]
swing_lows = df_slice[df_slice['ms_swing_low'] == 1]

# Plot swing markers
ax1.scatter(swing_highs.index, swing_highs['high'] * 1.01, label='Swing High', 
            marker='v', color='red', s=100) # s is marker size
ax1.scatter(swing_lows.index, swing_lows['low'] * 0.99, label='Swing Low', 
            marker='^', color='green', s=100)

ax1.set_ylabel('Price')
ax1.legend()
ax1.grid(True)

# Panel 2: Trend Regime
ax2 = axes[1]
ax2.plot(df_slice.index, df_slice['ms_trend_regime'], label='Trend Regime', 
         color='purple', drawstyle='steps-post')
ax2.set_title('Market Trend Regime')
ax2.set_yticks([-1, 0, 1])
ax2.set_yticklabels(['Down', 'Neutral', 'Up'])
ax2.set_ylabel('Regime')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()